In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [10]:
air_pollution <- read_csv("PRSA_Data_Aotizhongxin_20130301-20170228.csv")

Rows: 35064 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): wd, station
dbl (16): No, year, month, day, hour, PM2.5, PM10, SO2, NO2, CO, O3, TEMP, P...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,2013,3,1,0,4,4,4,7,300,77,-0.7,1023.0,-18.8,0,NNW,4.4,Aotizhongxin
2,2013,3,1,1,8,8,4,7,300,77,-1.1,1023.2,-18.2,0,N,4.7,Aotizhongxin
3,2013,3,1,2,7,7,5,10,300,73,-1.1,1023.5,-18.2,0,NNW,5.6,Aotizhongxin
4,2013,3,1,3,6,6,11,11,300,72,-1.4,1024.5,-19.4,0,NW,3.1,Aotizhongxin
5,2013,3,1,4,3,3,12,12,300,72,-2.0,1025.2,-19.5,0,N,2.0,Aotizhongxin
6,2013,3,1,5,5,5,18,18,400,66,-2.2,1025.6,-19.6,0,N,3.7,Aotizhongxin


In [95]:
install.packages("con2aqi")
library(con2aqi)
air_poll_nona <- na.omit(air_pollution)
air_poll_new <- air_poll_nona |>
    mutate(O3 = O3/1000, CO = CO/1000)|>
    filter(PM2.5 <= 500.4, PM10 <= 604, SO2 <= 1004, NO2 <= 2049, CO <= 50.4, O3 <= 0.604)
pm25_aqi <- con2aqi(pollutant = "pm25", con = air_poll_new$PM2.5)|>
    as.data.frame()
colnames(pm25_aqi) <- "pm25_aqi"
pm10_aqi <- con2aqi(pollutant = "pm10", con = air_poll_new$PM10)|>
    as.data.frame()
colnames(pm10_aqi) <- "pm10_aqi"
so2_aqi <- con2aqi(pollutant = "so2", con = air_poll_new$SO2)|>
    as.data.frame()
colnames(so2_aqi) <- "so2_aqi"
no2_aqi <- con2aqi(pollutant = "no2", con = air_poll_new$NO2)|>
    as.data.frame()
colnames(no2_aqi) <- "no2_aqi"
co_aqi <- con2aqi(pollutant = "co", con = air_poll_new$CO)|>
    as.data.frame()
colnames(co_aqi) <- "co_aqi"
o3_aqi <- con2aqi(pollutant = "o3", con = air_poll_new$O3, type = "1h")|>
    as.data.frame()
colnames(o3_aqi) <- "o3_aqi"

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,⋯,RAIN,wd,WSPM,station,pm25_aqi,pm10_aqi,so2_aqi,no2_aqi,co_aqi,o3_aqi
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2013,3,1,0,4,4,4,7,0.3,⋯,0,NNW,4.4,Aotizhongxin,17,4,6,7,4,82
2,2013,3,1,1,8,8,4,7,0.3,⋯,0,N,4.7,Aotizhongxin,34,8,6,7,4,82
3,2013,3,1,2,7,7,5,10,0.3,⋯,0,NNW,5.6,Aotizhongxin,30,7,8,10,4,80
4,2013,3,1,3,6,6,11,11,0.3,⋯,0,NW,3.1,Aotizhongxin,25,6,16,11,4,80
5,2013,3,1,4,3,3,12,12,0.3,⋯,0,N,2.0,Aotizhongxin,13,3,18,12,4,80
6,2013,3,1,5,5,5,18,18,0.4,⋯,0,N,3.7,Aotizhongxin,21,5,26,17,5,78
7,2013,3,1,6,3,3,18,32,0.5,⋯,0,NNE,2.5,Aotizhongxin,13,3,26,31,6,71
8,2013,3,1,7,3,6,19,41,0.5,⋯,0,NNW,3.8,Aotizhongxin,13,6,28,39,6,68
9,2013,3,1,8,3,6,16,43,0.5,⋯,0,NNW,4.1,Aotizhongxin,13,6,23,41,6,69


In [123]:
air_poll_aqi <- bind_cols(air_poll_new, pm25_aqi, pm10_aqi, so2_aqi, no2_aqi, co_aqi, o3_aqi)
air_poll_aqi <- air_poll_aqi |>
    select(-wd, -station)|>
    group_by(year, month, day) |>
    summarize(across(TEMP:o3_aqi, mean))|>
    rowwise()|>
    mutate(aqi = round(max(pm25_aqi:o3_aqi)))
air_poll_aqi

`summarise()` has grouped output by 'year', 'month'. You can override using the
`.groups` argument.


year,month,day,TEMP,PRES,DEWP,RAIN,WSPM,pm25_aqi,pm10_aqi,so2_aqi,no2_aqi,co_aqi,o3_aqi,aqi
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2013,3,1,1.3916667,1026.875,-18.7458333,0.000000000,3.254167,28.91667,10.37500,17.08333,21.91667,5.291667,76.79167,76
2013,3,2,0.6166667,1026.850,-15.9375000,0.000000000,1.479167,81.95833,34.66667,50.41667,64.45833,9.750000,63.12500,82
2013,3,3,5.5666667,1014.608,-12.3166667,0.000000000,1.658333,151.29167,84.08333,75.29167,79.62500,18.875000,58.95833,151
2013,3,4,10.3714286,1017.714,-11.6857143,0.000000000,2.361905,74.09524,38.66667,34.38095,49.14286,7.714286,72.19048,74
2013,3,5,6.2916667,1010.900,-7.5250000,0.000000000,1.129167,209.54167,115.62500,108.25000,101.79167,27.208333,78.66667,210
2013,3,6,7.3260870,1007.491,-4.1347826,0.000000000,1.486957,275.26087,160.56522,120.21739,109.26087,35.826087,59.08696,275
2013,3,7,9.2875000,1002.321,-2.0333333,0.000000000,1.558333,313.33333,190.12500,106.54167,109.66667,41.583333,66.79167,313
2013,3,8,10.3000000,999.595,-1.0850000,0.000000000,1.780000,262.65000,163.20000,82.35000,91.80000,28.550000,82.55000,263
2013,3,9,6.4000000,1004.119,-9.3190476,0.000000000,2.838095,123.00000,119.04762,48.00000,50.09524,15.333333,80.57143,123
